<div align="center">
<img style="display: block; margin: auto;" alt="photo" src="https://cdn.quantconnect.com/web/i/icon.png">

Quantconnect

Introduction to Financial Python
</div>

# 01 Data Types and Data Structures

# Introduction

This tutorial provides a basic introduction to the Python programming language. If you are new to Python, you should run the code snippets while reading this tutorial. If you are an advanced Python user, please feel free to skip this chapter.

# Basic Variable Types
The basic types of variables in Python are: strings, integers, floating point numbers and booleans.

Strings in python are identified as a contiguous set of characters represented in either single quotes (' ') or double quotes (" ").


In [ ]:
my_string1 = 'Welcome to'
my_string2 = "QuantConnect"
print(my_string1 + ' ' + my_string2)

Welcome to QuantConnect


In [ ]:
my_string1 = 'Tutorial'
my_string2 = "primer laboratorio"
print(my_string1 + ' ' + my_string2)

Tutorial primer laboratorio


An integer is a round number with no values after the decimal point.

In [ ]:
my_int = 10
print(my_int)
print(type(my_int))

10
<class 'int'>


In [ ]:
my_int = 25
print(my_int)
print(type(my_int))

25
<class 'int'>


The built-in function int() can convert a string into an integer.

In [ ]:
my_string = "100"
print(type(my_string))
my_int = int(my_string)
print(type(my_int))

<class 'str'>
<class 'int'>


In [ ]:
my_string = "555"
print(type(my_string))
my_int = int(my_string)
print(type(my_int))

<class 'str'>
<class 'int'>


A floating point number, or a float, is a real number in mathematics. In Python we need to include a value after a decimal point to define it as a float

In [ ]:
my_string = "100"
my_float = float(my_string)
print(type(my_float))

<class 'float'>


In [ ]:
my_string = "500"
my_float = float(my_string)
print(type(my_float))

<class 'float'>


As you can see above, if we don't include a decimal value, the variable would be defined as an integer. The built-in function float() can convert a string or an integer into a float.

In [ ]:
my_bool = False
print(my_bool)
print(type(my_bool))

False
<class 'bool'>


In [ ]:
my_bool = True
print(my_bool)
print(type(my_bool))

True
<class 'bool'>


A boolean, or bool, is a binary variable. Its value can only be True or False. It is useful when we do some logic operations, which would be covered in our next chapter.

In [ ]:
print("Addition ", 1+1)
print("Subtraction ", 5-2)
print("Multiplication ", 2*3)
print("Division ", 10/2)
print('exponent', 2**3)

Addition  2
Subtraction  3
Multiplication  6
Division  5.0
exponent 8


In [ ]:
print("Addition ", 2+1)
print("Subtraction ", 6-2)
print("Multiplication ", 3*3)
print("Division ", 20/2)
print('exponent', 3**3)

Addition  3
Subtraction  4
Multiplication  9
Division  10.0
exponent 27


# Basic Math Operations

The basic math operators in python are demonstrated below:

In [ ]:
print(1/3)
print(1.0/3)

0.3333333333333333
0.3333333333333333


In [ ]:
print(3/3)
print(1.0/4)

1.0
0.25


# Data Collections

## List
A list is an ordered collection of values. A list is mutable, which means you can change a list's value without changing the list itself. Creating a list is simply putting different comma-separated values between square brackets.

In [ ]:
my_list = ['Quant', 'Connect', 1,2,3]
print(my_list)

['Quant', 'Connect', 1, 2, 3]


In [ ]:
my_list = ['primer', 'tutorial', '2021',1,2]
print(my_list)

['primer', 'tutorial', '2021', 1, 2]


The values in a list are called "elements". We can access list elements by indexing. Python index starts from 0. So if you have a list of length n, the index of the first element will be 0, and that of the last element will be n − 1. By the way, the length of a list can be obtained by the built-in function len().

In [ ]:
my_list = ['Quant', 'Connect', 1,2,3]
print(len(my_list))
print(my_list[0])
print(my_list[len(my_list) -1])

5
Quant
3


In [ ]:
my_list = ['Quant', 'Connect', 1,2,3,4]
print(len(my_list))
print(my_list[3])
print(my_list[len(my_list) -1])

6
2
4


You can also change the elements in the list by accessing an index and assigning a new value.

In [ ]:
my_list = ['Quant','Connect',1,2,3]
my_list[2] = 'go'
print(my_list)

['Quant', 'Connect', 'go', 2, 3]


In [ ]:
my_list = ['Quant','Connect',1,2,3]
my_list[0] = 'go'
print(my_list)

['go', 'Connect', 1, 2, 3]


A list can also be sliced with a colon:

In [ ]:
my_list = ['Quant','Connect',1,2,3]
print(my_list[1:3])

['Connect', 1]


In [ ]:
my_list = ['Quant','Connect',1,2,3]
print(my_list[1:4])

['Connect', 1, 2]


The slice starts from the first element indicated, but excludes the last element indicated. Here we select all elements starting from index 1, which refers to the second element:

In [ ]:
print(my_list[1:])

['Connect', 1, 2, 3]


In [ ]:
print(my_list[3:])

[2, 3]


And all elements up to but excluding index 3:

In [ ]:
print(my_list[:3])

['Quant', 'Connect', 1]


In [ ]:
print(my_list[:2])

['Quant', 'Connect']


If you wish to add or remove an element from a list, you can use the append() and remove() methods for lists as follows:

In [ ]:
my_list = ['Hello', 'Quant']
my_list.append('Hello')
print(my_list)



['Hello', 'Quant', 'Hello']


In [ ]:
my_list = ['Hello', 'Quant']
my_list.append('Prueba')
print(my_list)

['Hello', 'Quant', 'Prueba']


In [ ]:
my_list.remove('Hello')
print(my_list)

['Quant', 'Hello']


In [ ]:
my_list.remove('Hello')
print(my_list)

['Quant', 'Prueba']


When there are repeated instances of "Hello", the first one is removed.

## Tuple
A tuple is a data structure type similar to a list. The difference is that a tuple is immutable, which means you can't change the elements in it once it's defined. We create a tuple by putting comma-separated values between parentheses.

In [ ]:
my_tuple = ('Welcome','to','QuantConnect')

Just like a list, a tuple can be sliced by using index.

In [ ]:
my_tuple = ('Welcome','to','QuantConnect')
print(my_tuple[1:])

('to', 'QuantConnect')


In [ ]:
my_tuple = ('Welcome','to','QuantConnect')
print(my_tuple[2:])

('QuantConnect',)


## Set
A set is an **unordered**  collection with **no duplicate** elements. The built-in function **set()** can be used to create sets.

In [ ]:
stock_list = ['AAPL','GOOG','IBM','AAPL','IBM','FB','F','GOOG']
stock_set = set(stock_list)
print(stock_set)

{'FB', 'F', 'GOOG', 'IBM', 'AAPL'}


In [ ]:
stock_list = ['add','repo','add','AAPL','repo','FB','F','GOOG']
stock_set = set(stock_list)
print(stock_set)

{'GOOG', 'AAPL', 'FB', 'repo', 'add', 'F'}


Set is an easy way to remove duplicate elements from a list.

##Dictionary
A dictionary is one of the most important data structures in Python. Unlike sequences which are indexed by integers, dictionaries are indexed by keys which can be either strings or floats.

A dictionary is an **unordered** collection of key : value pairs, with the requirement that the keys are unique. We create a dictionary by placing a comma-separated list of key : value pairs within the braces.

In [ ]:
my_dic = {'AAPL':'AAPLE', 'FB':'FaceBook', 'GOOG':'Alphabet'}

In [ ]:
print(my_dic['FB'])

FaceBook


After defining a dictionary, we can access any value by indicating its key in brackets.

In [ ]:
my_dic['GOOG'] = 'Alphabet Company'
print(my_dic['GOOG'])

Alphabet Company


We can also change the value associated with a specified key:

In [ ]:
print(my_dic.keys())

dict_keys(['AAPL', 'FB', 'GOOG', 'PRUEBA'])


The built-in method of the dictionary object dict.keys() returns a list of all the keys used in the dictionary.

In [ ]:
my_dic = {'Manzana':'AAPLE', 'naranja':'ORANGE', 'perro':'DOG'}
print(my_dic['Manzana'])

AAPLE


In [ ]:
print(my_dic.keys())

dict_keys(['Manzana', 'naranja', 'perro'])


# Common String Operations
A string is an immutable sequence of characters. It can be sliced by index just like a tuple:

In [ ]:
my_str = 'Welcome to QuantConnect'
print(my_str[8:])

to QuantConnect


In [ ]:
my_str = 'Una palabra demasiado larga'
print(my_str[11:])

 demasiado larga


There are many methods associated with strings. We can use string.count() to count the occurrences of a character in a string, use string.find() to return the index of a specific character, and use string.replace() to replace characters

In [ ]:
print('Counting the number of e appears in this sentence'.count('e'))
print('The first time e appears in this sentence'.find('e'))
print('all the a in this sentence now becomes e'.replace('a','e'))

7
2
ell the e in this sentence now becomes e


In [ ]:
print('tres tristes tigres tragaban trigo en un trigal'.count('e'))
print('la primer vez de los tigres'.find('m'))
print('tres tristes tigres tragaban trigo en un trigal'.replace('t','s'))

4
6
sres srisses sigres sragaban srigo en un srigal


The most commonly used method for strings is string.split(). This method will split the string by the indicated character and return a list:

In [ ]:
Time = '2016-04-01 09:43:00'
splited_list = Time.split(' ')
date = splited_list[0]
time = splited_list[1]
print(date, time)
hour = time.split(':')[0]
print(hour)

2016-04-01 09:43:00
09


In [ ]:
Time = '2021-10-16 10:43:00 '
splited_list = Time.split(' ')
date = splited_list[0]
time = splited_list[1]
print(date, time)
hour = time.split(':')[0]
print(hour)

2021-10-16 10:43:00
10


We can replace parts of a string by our variable. This is called string formatting.

In [ ]:
my_time = 'Hour: {}, Minute:{}'.format('09','43')
print(my_time)

Hour: 09, Minute:43


In [ ]:
my_time = 'Hour: {}, Minute:{}'.format('10','43')
print(my_time)

Hour: 10, Minute:43


Another way to format a string is to use the % symbol.

In [ ]:
print('the pi number is %f'%3.14)
print('%s to %s'%('Welcome','Quantconnect'))

the pi number is 3.140000
Welcome to Quantconnect


In [ ]:
print('the pi number is %f'%3.14)
print('%s como %s'%('Hola','esta?'))

the pi number is 3.140000
Hola como esta?


# Summary

Weave seen the basic data types and data structures in Python. It's important to keep practicing to become familiar with these data structures. In the next tutorial, we will cover for and while loops and logical operations in Python.

<div align="center">
<img style="display: block; margin: auto;" alt="photo" src="https://cdn.quantconnect.com/web/i/icon.png">

Quantconnect

Introduction to Financial Python
</div>

# 02 Logical Operations and Loops

# Introduction
We discussed the basic data types and data structures in Python in the last tutorial. This chapter covers logical operations and loops in Python, which are very common in programming.

# Logical Operations
Like most programming languages, Python has comparison operators:

In [63]:
print(1 == 0)
print(1 == 1)
print(1 != 0)
print(5 >= 5)
print(5 >= 6)

False
True
True
True
False


In [64]:
print(4 == 0)
print(2 == 1)
print(1 != 0)
print(5 >= 10)
print(5 >= 5)

False
False
True
False
True


Each statement above has a boolean value, which must be either True or False, but not both.

We can combine simple statements P and Q to form complex statements using logical operators:

- The statement "P and Q" is true if both P and Q are true, otherwise it is false.
- The statement "P or Q" is false if both P and Q are false, otherwise it is true.
- The statement "not P" is true if P is false, and vice versa.

In [66]:
print(2 > 1 and 3 > 2)
print(2 > 1 and 3 < 2) 
print(2 > 1 or 3 < 2)
print(2 < 1 and 3 < 2)

True
False
True
False


In [65]:
print(2 > 1 and 1 > 2)
print(2 > 1 and 4 < 2) 
print(4 > 1 or 3 < 2)
print(0 < 1 and 3 < 2)

False
False
True
False


When dealing with a very complex logical statement that involves in several statements, we can use brackets to separate and combine them.

In [67]:
print((4 > 2 or 4 < 3) and (1!=3 and 6>3) and not ( 7 < 2 or 1 < 3 and (3!=3 and 6>3)))
print(4 > 2 or 4 < 3 and (1!=3 and 6>3) and not ( 7 < 2 or 1 < 3 and (3!=3 and 4>3)))

True
True


Comparing the above two statements, we can see that it's wise to use brackets when we make a complex logical statement.

# If Statement
An if statement executes a segment of code only if its condition is true. A standard if statement consists of 3 segments: if, elif and else.

```python
if statement1:
    # if the statement1 is true, execute the code here.
    # code.....
    # code.....
elif statement2:
    # if the statement 1 is false, skip the codes above to this part.
    # code......
    # code......
else:
    # if none of the above statements is True, skip to this part
    # code......
```

An if statement doesn't necessarily has elif and else part. If it's not specified, the indented block of code will be executed when the condition is true, otherwise the whole if statement will be skipped.

In [ ]:
i = 0
if i == 0:
    print('i==0 is True')

i==0 is True


In [70]:
i = 19
if i == 19:
    print('i==0 is True')

i==0 is True


As we mentioned above, we can write some complex statements here:

In [72]:
p = 1 > 0
q = 2 > 3
if p and q:
    print('p and q is true')
elif p and not q:
    print('q is false')
elif q and not p:
    print('p is false')
else:
    print('None of p and q is true')

q is false


In [73]:
p = 7 > 0
q = 1 > 3
if p and q:
    print('p and q is true')
elif p and not q:
    print('q is false')
elif q and not p:
    print('p is false')
else:
    print('None of p and q is true')

q is false


# Loop Structure
Loops are an essential part of programming. The "for" and "while" loops run a block of code repeatedly.

## While Loop
A "while" loop will run repeatedly until a certain condition has been met.

In [74]:
i = 0
while i < 5:
    print(i)
    i += 1  

0
1
2
3
4


In [75]:
i = 2
while i < 5:
    print(i)
    i += 1  

2
3
4


When making a while loop, we need to ensure that something changes from iteration to iteration so that the while loop will terminate, otherwise, it will run forever. Here we used i += 1 (short for i = i + 1) to make i larger after each iteration. This is the most commonly used method to control a while loop.

## For Loop
A "for" loop will iterate over a sequence of value and terminate when the sequence has ended.

In [76]:
for i in [1,2,3,4,5]:
    print(i)

1
2
3
4
5


In [77]:
for i in [1,2,3]:
    print(i)

1
2
3


We can also add if statements in a for loop. Here is a real example from our pairs trading algorithm:

In [78]:
stocks = ['AAPL','GOOG','IBM','FB','F','V', 'G', 'GE']
selected = ['AAPL','IBM']
new_list = []
for i in stocks:
    if i not in selected:
        new_list.append(i)
print(stocks)

['AAPL', 'GOOG', 'IBM', 'FB', 'F', 'V', 'G', 'GE']


In [87]:
stocks = ['AAPL','GOOG','IBM','FB','F','V', 'G', 'GE']
selected = ['AAPL','IBM','GOOG']
new_list = []
for i in stocks:
    if i not in selected:
        new_list.append(i)
print(stocks)
print(new_list)


['AAPL', 'GOOG', 'IBM', 'FB', 'F', 'V', 'G', 'GE']
['FB', 'F', 'V', 'G', 'GE']


Here we iterated all the elements in the list 'stocks'. Later in this chapter, we will introduce a smarter way to do this, which is just a one-line code.

## Break and continue
These are two commonly used commands in a for loop. If "break" is triggered while a loop is executing, the loop will terminate immediately:

In [88]:
stocks = ['AAPL','GOOG','IBM','FB','F','V', 'G', 'GE']
for i in stocks:
    print(i)
    if i == 'FB':
        break

AAPL
GOOG
IBM
FB


In [89]:
stocks = ['AAPL','GOOG','IBM','FB','F','V', 'G', 'GE']
for i in stocks:
    print(i)
    if i == 'G':
        break

AAPL
GOOG
IBM
FB
F
V
G


The "continue" command tells the loop to end this iteration and skip to the next iteration:

In [91]:
stocks = ['AAPL','GOOG','IBM','FB','F','V', 'G', 'GE']
for i in stocks:
    if i == 'FB':
        continue
    print(i)

AAPL
GOOG
IBM
F
V
G
GE


In [93]:
stocks = ['AAPL','GOOG','IBM','FB','F','V', 'G', 'GE']
for i in stocks:
    if i == 'GOOG':
        continue
    print(i)

AAPL
IBM
FB
F
V
G
GE


# List Comprehension
List comprehension is a Pythonic way to create lists. Common applications are to make new lists where each element is the result of some operations applied to each member of another sequence. For example, if we want to create a list of squares using for loop:

In [94]:
squares = []
for i in [1,2,3,4,5]:
    squares.append(i**2)
print(squares)

[1, 4, 9, 16, 25]


In [95]:
squares = []
for i in [10,20,30,40,50]:
    squares.append(i**2)
print(squares)

[100, 400, 900, 1600, 2500]


Using list comprehension:

In [96]:
list = [1,2,3,4,5]
squares = [x**2 for x in list]
print(squares)

[1, 4, 9, 16, 25]


In [97]:
list = [1,2,3,4,5]
squares = [x**3 for x in list]
print(squares)

[1, 8, 27, 64, 125]


Recall the example above where we used a for loop to select stocks. Here we use list comprehension:

In [98]:
stocks = ['AAPL','GOOG','IBM','FB','F','V', 'G', 'GE']
selected = ['AAPL','IBM']
new_list = [x for x in stocks if x in selected]
print(new_list)

['AAPL', 'IBM']


In [99]:
stocks = ['AAPL','GOOG','IBM','FB','F','V', 'G', 'GE']
selected = ['GOOG','FB']
new_list = [x for x in stocks if x in selected]
print(new_list)

['GOOG', 'FB']


A list comprehension consists of square brackets containing an expression followed by a "for" clause, and possibly "for" or "if" clauses. For example:


In [101]:
print([(x, y) for x in [1,2,3] for y in [3,1,4] if x != y])
print([str(x)+' vs '+str(y) for x in ['AAPL','GOOG','IBM','FB'] for y in ['F','V','G','GE'] if x!=y])

[(1, 3), (1, 4), (2, 3), (2, 1), (2, 4), (3, 1), (3, 4)]
['AAPL vs F', 'AAPL vs V', 'AAPL vs G', 'AAPL vs GE', 'GOOG vs F', 'GOOG vs V', 'GOOG vs G', 'GOOG vs GE', 'IBM vs F', 'IBM vs V', 'IBM vs G', 'IBM vs GE', 'FB vs F', 'FB vs V', 'FB vs G', 'FB vs GE']


In [102]:
print([(x, y) for x in [4,5,6] for y in [7,8,9] if x != y])
print([str(x)+' vs '+str(y) for x in ['abc','def','ghi','jk'] for y in ['1','2','3','4'] if x!=y])

[(4, 7), (4, 8), (4, 9), (5, 7), (5, 8), (5, 9), (6, 7), (6, 8), (6, 9)]
['abc vs 1', 'abc vs 2', 'abc vs 3', 'abc vs 4', 'def vs 1', 'def vs 2', 'def vs 3', 'def vs 4', 'ghi vs 1', 'ghi vs 2', 'ghi vs 3', 'ghi vs 4', 'jk vs 1', 'jk vs 2', 'jk vs 3', 'jk vs 4']


<div align="center">
<img style="display: block; margin: auto;" alt="photo" src="https://cdn.quantconnect.com/web/i/icon.png">

Quantconnect

Introduction to Financial Python
</div>

# 03 Functions and Objective-Oriented Programming

# Introduction

In the last tutorial we introduced logical operations, loops and list comprehension. We will introduce functions and object-oriented programming in this chapter, which will enable us to build complex algorithms in more flexible ways.

# Functions
A function is a reusable block of code. We can use a function to output a value, or do anything else we want. We can easily define our own function by using the keyword "def".

In [ ]:
def product(x,y):
    return x*y
print(product(2,3))
print(product(5,10))

In [104]:
def product(x,y):
    return x*y
print(product(3,3))
print(product(1,10))

9
10


The keyword "def" is followed by the function name and the parenthesized list of formal parameters. The statements that form the body of the function start at the next line, and must be indented. The product() function above has "x" and "y" as its parameters. A function doesn't necessarily have parameters:

In [106]:
def say_hi():
    print('Welcome to QuantConnect')
say_hi()

Welcome to QuantConnect


In [105]:
def say_hi():
    print('laboratorio 1')
say_hi()

laboratorio 1


# Built-in Function
**range()** is a function that creates a list containing an arithmetic sequence. It's often used in for loops. The arguments must be integers. If the "step" argument is omitted, it defaults to 1.

In [107]:
print(range(10))
print(range(1,11))
print(range(1,11,2))

range(0, 10)
range(1, 11)
range(1, 11, 2)


In [108]:
print(range(5))
print(range(1,5))
print(range(1,2,4))

range(0, 5)
range(1, 5)
range(1, 2, 4)


**len()** is another function used together with range() to create a for loop. This function returns the length of an object. The argument must be a sequence or a collection.

In [109]:
tickers = ['AAPL','GOOG','IBM','FB','F','V', 'G', 'GE']
print('The length of tickers is {}'.format(len(tickers)))
for i in range(len(tickers)):
    print(tickers[i])

The length of tickers is 8
AAPL
GOOG
IBM
FB
F
V
G
GE


In [111]:
tickers = ['1','2','3','4','5','5', '6', '7']
print('The length of tickers is {}'.format(len(tickers)))
for i in range(len(tickers)):
    print(tickers[i])

The length of tickers is 8
1
2
3
4
5
5
6
7


Note: If you want to print only the tickers without those numbers, then simply write "for ticker in tickers: print ticker"

**map(**) is a function that applies a specific function to every item of a sequence or collection, and returns a list of the results.

In [1]:
tickers = ['AAPL','GOOG','IBM','FB','F','V', 'G', 'GE']
print(list(map(len,tickers)))

[4, 4, 3, 2, 1, 1, 1, 2]


In [3]:
tickers = ['AL','GOG','IM','FB','F','V', 'G', 'GE']
print(list(map(len,tickers)))

[2, 3, 2, 2, 1, 1, 1, 2]


The **lambda operator** is a way to create small anonymous functions. These functions are just needed where they have been created. For example:

In [4]:
list(map(lambda x: x**2, range(10)))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [5]:
list(map(lambda x: x**3, range(5)))

[0, 1, 8, 27, 64]

map() can be applied to more than one list. The lists have to have the same length.

In [6]:
list(map(lambda x, y: x+y, [1,2,3,4,5],[5,4,3,2,1]))

[6, 6, 6, 6, 6]

In [7]:
list(map(lambda x, y: x+y, [1,2,3,4,5],[7,8,9,10,11]))

[8, 10, 12, 14, 16]

sorted() takes a list or set and returns a new sorted list


In [8]:
sorted([5,2,3,4,1])

[1, 2, 3, 4, 5]

In [9]:
sorted([55,444,1,4,5])

[1, 4, 5, 55, 444]

We can add a "key" parameter to specify a function to be called on each list element prior to making comparisons. For example:

In [10]:
price_list = [('AAPL',144.09),('GOOG',911.71),('MSFT',69),('FB',150),('WMT',75.32)]
sorted(price_list, key = lambda x: x[1])

[('MSFT', 69), ('WMT', 75.32), ('AAPL', 144.09), ('FB', 150), ('GOOG', 911.71)]

In [11]:
price_list = [('AAPL',1.09),('GOOG',11.71),('MSFT',9),('FB',50),('WMT',5.32)]
sorted(price_list, key = lambda x: x[1])

[('AAPL', 1.09), ('WMT', 5.32), ('MSFT', 9), ('GOOG', 11.71), ('FB', 50)]

By default the values are sorted by ascending order. We can change it to descending by adding an optional parameter "reverse'.

In [12]:
price_list = [('AAPL',144.09),('GOOG',911.71),('MSFT',69),('FB',150),('WMT',75.32)]
sorted(price_list, key = lambda x: x[1],reverse = True)

[('GOOG', 911.71), ('FB', 150), ('AAPL', 144.09), ('WMT', 75.32), ('MSFT', 69)]

In [14]:
price_list = [('AAPL',1.09),('GOOG',11.71),('MSFT',6),('FB',50),('WMT',5.32)]
sorted(price_list, key = lambda x: x[1],reverse = True)

[('FB', 50), ('GOOG', 11.71), ('MSFT', 6), ('WMT', 5.32), ('AAPL', 1.09)]

Lists also have a function list.sort(). This function takes the same "key" and "reverse" arguments as sorted(), but it doesn't return a new list.

In [15]:
price_list = [('AAPL',144.09),('GOOG',911.71),('MSFT',69),('FB',150),('WMT',75.32)]
price_list.sort(key = lambda x: x[1])
print(price_list)

[('MSFT', 69), ('WMT', 75.32), ('AAPL', 144.09), ('FB', 150), ('GOOG', 911.71)]


In [16]:
price_list = [('AAPL',14.09),('GOOG',1.71),('MSFT',6),('FB',1),('WMT',5.32)]
price_list.sort(key = lambda x: x[1])
print(price_list)

[('FB', 1), ('GOOG', 1.71), ('WMT', 5.32), ('MSFT', 6), ('AAPL', 14.09)]


# Object-Oriented Programming
Python is an object-oriented programming language. It's important to understand the concept of "objects" because almost every kind of data from QuantConnect API is an object.

## Class
A class is a type of data, just like a string, float, or list. When we create an object of that data type, we call it an instance of a class.

In Python, everything is an object - everything is an instance of some class. The data stored inside an object are called attributes, and the functions which are associated with the object are called methods.

For example, as mentioned above, a list is an object of the "list" class, and it has a method list.sort().

We can create our own objects by defining a class. We would do this when it's helpful to group certain functions together. For example, we define a class named "Stock" here:

In [17]:
class stock:
    def __init__(self, ticker, open, close, volume):
        self.ticker = ticker
        self.open = open
        self.close = close
        self.volume = volume
        self.rate_return = float(close)/open - 1
 
    def update(self, open, close):
        self.open = open
        self.close = close
        self.rate_return = float(self.close)/self.open - 1
 
    def print_return(self):
        print(self.rate_return)

In [48]:
apple = stock('AAPL', 143.69, 144.09, 20109375)
google = stock('GOOG', 898.7, 911.7, 1561616)

In [49]:
apple.ticker
google.print_return()
google.update(912.8,913.4)
google.print_return()

0.014465338822744034
0.0006573181419806673


In [46]:
apple = stock('AAPL', 153.69, 104.09, 201093)
google = stock('GOOG', 198.7, 111.7, 561616)

In [50]:
apple.ticker
google.print_return()
google.update(340.8,358.4)
google.print_return()

0.0006573181419806673
0.05164319248826277


By calling the update() function, we updated the open and close prices of a stock. Please note that when we use the attributes or call the methods **inside a class**, we need to specify them as self.attribute or self.method(), otherwise Python will deem them as global variables and thus raise an error.

We can add an attribute to an object anywhere:

In [51]:
apple.ceo = 'Tim Cook'
apple.ceo

'Tim Cook'

In [52]:
apple.ceo = 'Tim'
apple.ceo

'Tim'

We can check what names (i.e. attributes and methods) are defined on an object using the dir() function:

In [53]:
dir(apple)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'ceo',
 'close',
 'open',
 'print_return',
 'rate_return',
 'ticker',
 'update',
 'volume']

In [54]:
dir(google)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'close',
 'open',
 'print_return',
 'rate_return',
 'ticker',
 'update',
 'volume']

## Inheritance
Inheritance is a way of arranging classes in a hierarchy from the most general to the most specific. A "child" class is a more specific type of a "parent" class because a child class will inherit all the attribues and methods of its parent. For example, we define a class named "Child" which inherits "Stock":

In [55]:
class child(stock):
    def __init__(self,name):
        self.name = name

In [57]:
aa = child('aa')
print(aa.name)
aa.update(100,102)
print(aa.open)
print(aa.close)
print(aa.print_return())

aa
100
102
0.020000000000000018
None


In [59]:
aa = child('33')
print(aa.name)
aa.update(130,189)
print(aa.open)
print(aa.close)
print(aa.print_return())

33
130
189
0.4538461538461538
None


<div align="center">
<img style="display: block; margin: auto;" alt="photo" src="https://cdn.quantconnect.com/web/i/icon.png">

Quantconnect

Introduction to Financial Python
</div>

# 04 NumPy and Basic Pandas

# Introduction

Now that we have introduced the fundamentals of Python, it's time to learn about NumPy and Pandas.

# NumPy
NumPy is the core library for scientific computing in Python. It provides a high-performance multidimensional array object, and tools for working with these arrays. It also has strong integration with Pandas, which is another powerful tool for manipulating financial data.

Python packages like NumPy and Pandas contain classes and methods which we can use by importing the package:

In [60]:
import numpy as np

price_list = [143.73, 145.83, 143.68, 144.02, 143.5, 142.62]
price_array = np.array(price_list)
print(price_array, type(price_array))

[143.73 145.83 143.68 144.02 143.5  142.62] <class 'numpy.ndarray'>


In [61]:
import numpy as np

price_list = [43.73, 45.83, 43.68, 44.02, 43.5, 12.62]
price_array = np.array(price_list)
print(price_array, type(price_array))

[43.73 45.83 43.68 44.02 43.5  12.62] <class 'numpy.ndarray'>


Notice that the type of array is "ndarray" which is a multi-dimensional array. If we pass np.array() a list of lists, it will create a 2-dimensional array.

In [94]:
Ar = np.array([[1,3],[2,4]])
print(Ar, type(Ar))

[[1 3]
 [2 4]] <class 'numpy.ndarray'>


In [95]:
print(Ar.shape)

(2, 2)


In [96]:
print(Ar[0])
print(Ar[1])

[1 3]
[2 4]


In [97]:
print('the first column: ', Ar[:,0])
print('the second column: ', Ar[:,1])

the first column:  [1 2]
the second column:  [3 4]


In [ ]:
ejemplo

In [89]:
Ar = np.array([[1,0],[0,1]])
print(Ar, type(Ar))

[[1 0]
 [0 1]] <class 'numpy.ndarray'>


In [90]:
print(Ar.shape)

(2, 2)


In [91]:
print(Ar[0])
print(Ar[1])

[1 0]
[0 1]


In [92]:
print('the first column: ', Ar[:,0])
print('the second column: ', Ar[:,1])

the first column:  [1 0]
the second column:  [0 1]


## Array Functions
Some functions built in NumPy that allow us to perform calculations on arrays. For example, we can apply the natural logarithm to each element of an array:

In [99]:
print(np.log(price_array))

[3.77803437 3.8249389  3.77689033 3.78464408 3.77276094 2.53528286]


In [100]:
print(np.mean(price_array))
print(np.std(price_array))
print(np.sum(price_array))
print(np.max(price_array))

38.89666666666667
11.77720208236612
233.38000000000002
45.83


The functions above return the mean, standard deviation, total and maximum value of an array.

# Pandas
Pandas is one of the most powerful tools for dealing with financial data. 

First we need to import Pandas:

In [102]:
import pandas as pd

price = [143.73, 145.83, 143.68, 144.02, 143.5, 142.62]
s = pd.Series(price)
s

0    143.73
1    145.83
2    143.68
3    144.02
4    143.50
5    142.62
dtype: float64

In [103]:
s = pd.Series(price,index = ['a','b','c','d','e','f'])
s

a    143.73
b    145.83
c    143.68
d    144.02
e    143.50
f    142.62
dtype: float64

In [104]:
s.index = [6,5,4,3,2,1]
s

6    143.73
5    145.83
4    143.68
3    144.02
2    143.50
1    142.62
dtype: float64

In [105]:
print(s[1:])
print(s[:-2])

5    145.83
4    143.68
3    144.02
2    143.50
1    142.62
dtype: float64
6    143.73
5    145.83
4    143.68
3    144.02
dtype: float64


In [106]:
print(s[4])
s[4] = 0
print(s)

143.68
6    143.73
5    145.83
4      0.00
3    144.02
2    143.50
1    142.62
dtype: float64


In [107]:
s = pd.Series(price, name = 'Apple Price List')
print(s)
print(s.name)

0    143.73
1    145.83
2    143.68
3    144.02
4    143.50
5    142.62
Name: Apple Price List, dtype: float64
Apple Price List


In [108]:
print(s.describe())

count      6.000000
mean     143.896667
std        1.059711
min      142.620000
25%      143.545000
50%      143.705000
75%      143.947500
max      145.830000
Name: Apple Price List, dtype: float64


EJEMPLO

In [118]:
price = [3, 45.83, 43.68, 44.02, 5, 2.62]
s = pd.Series(price)
s

0     3.00
1    45.83
2    43.68
3    44.02
4     5.00
5     2.62
dtype: float64

In [119]:
s = pd.Series(price,index = ['a','b','c','d','e','f'])
s


a     3.00
b    45.83
c    43.68
d    44.02
e     5.00
f     2.62
dtype: float64

In [120]:
s.index = [6,5,4,3,2,1]
s

6     3.00
5    45.83
4    43.68
3    44.02
2     5.00
1     2.62
dtype: float64

In [121]:
print(s[1:])
print(s[:-2])

5    45.83
4    43.68
3    44.02
2     5.00
1     2.62
dtype: float64
6     3.00
5    45.83
4    43.68
3    44.02
dtype: float64


In [122]:
print(s[4])
s[4] = 0
print(s)

43.68
6     3.00
5    45.83
4     0.00
3    44.02
2     5.00
1     2.62
dtype: float64


In [123]:
s = pd.Series(price, name = 'Apple Price List')
print(s)
print(s.name)

0     3.00
1    45.83
2    43.68
3    44.02
4     5.00
5     2.62
Name: Apple Price List, dtype: float64
Apple Price List


In [124]:
print(s.describe())

count     6.000000
mean     24.025000
std      22.466652
min       2.620000
25%       3.500000
50%      24.340000
75%      43.935000
max      45.830000
Name: Apple Price List, dtype: float64


## Time Index
Pandas has a built-in function specifically for creating date indices: pd.date_range(). We use it to create a new index for our Series:

In [125]:
time_index = pd.date_range('2017-01-01',periods = len(s),freq = 'D')
print(time_index)
s.index = time_index
print(s)

DatetimeIndex(['2017-01-01', '2017-01-02', '2017-01-03', '2017-01-04',
               '2017-01-05', '2017-01-06'],
              dtype='datetime64[ns]', freq='D')
2017-01-01     3.00
2017-01-02    45.83
2017-01-03    43.68
2017-01-04    44.02
2017-01-05     5.00
2017-01-06     2.62
Freq: D, Name: Apple Price List, dtype: float64


In [126]:
s.index = [6,5,4,3,2,1]
print(s)
print(s[1])

6     3.00
5    45.83
4    43.68
3    44.02
2     5.00
1     2.62
Name: Apple Price List, dtype: float64
2.62


In [127]:
print(s.iloc[1])

45.83


In [128]:
s.index = time_index
print(s['2017-01-03'])

43.68


In [129]:
print(s['2017-01-02':'2017-01-05'])

2017-01-02    45.83
2017-01-03    43.68
2017-01-04    44.02
2017-01-05     5.00
Freq: D, Name: Apple Price List, dtype: float64


In [130]:
print(s[s < np.mean(s)] )
print([(s > np.mean(s)) & (s < np.mean(s) + 1.64*np.std(s))])

2017-01-01    3.00
2017-01-05    5.00
2017-01-06    2.62
Name: Apple Price List, dtype: float64
[2017-01-01    False
2017-01-02     True
2017-01-03     True
2017-01-04     True
2017-01-05    False
2017-01-06    False
Freq: D, Name: Apple Price List, dtype: bool]


EJEMPLO

In [131]:
time_index = pd.date_range('2021-10-15',periods = len(s),freq = 'D')
print(time_index)
s.index = time_index
print(s)

DatetimeIndex(['2021-10-15', '2021-10-16', '2021-10-17', '2021-10-18',
               '2021-10-19', '2021-10-20'],
              dtype='datetime64[ns]', freq='D')
2021-10-15     3.00
2021-10-16    45.83
2021-10-17    43.68
2021-10-18    44.02
2021-10-19     5.00
2021-10-20     2.62
Freq: D, Name: Apple Price List, dtype: float64


In [132]:
s.index = [6,5,4,3,2,1]
print(s)
print(s[1])

6     3.00
5    45.83
4    43.68
3    44.02
2     5.00
1     2.62
Name: Apple Price List, dtype: float64
2.62


In [133]:
print(s.iloc[1])

45.83


In [135]:
s.index = time_index
print(s['2021-10-15'])

3.0


In [137]:
print(s['2021-10-10':'2021-10-15'])

2021-10-15    3.0
Freq: D, Name: Apple Price List, dtype: float64


In [138]:
print(s[s < np.mean(s)] )
print([(s > np.mean(s)) & (s < np.mean(s) + 1.64*np.std(s))])

2021-10-15    3.00
2021-10-19    5.00
2021-10-20    2.62
Name: Apple Price List, dtype: float64
[2021-10-15    False
2021-10-16     True
2021-10-17     True
2021-10-18     True
2021-10-19    False
2021-10-20    False
Freq: D, Name: Apple Price List, dtype: bool]
